**Note: This is my EDA.  It's not pretty or nice and probably has some mistakes along the way.  Check out the associated `.py` file for a linted/formatted version of the solution.**

In [1]:
from pathlib import Path
import sys

import polars as pl

# Gross hack to get jupyter to find hod stuff.
sys.path.append(str(Path(Path.cwd().parent)))
from hod.utils import get_data

# The Contractor

(JAMES: Emphasis below mine when I think something will be useful.)

REF: https://hanukkah.bluebird.sh/5784/2/

Thanks to your help, Sarah called the investigator that afternoon. The investigator went directly to the cleaners to see if they could get any more information about the unclaimed rug.

While they were out, Sarah said, “I tried cleaning the rug myself, but there was this snail on it that always seemed to leave a trail of slime behind it. I spent a few hours cleaning it, and the next day the slime trail was back.”

When the investigator returned, they said, “Apparently, this cleaner had a special projects program, where they outsourced challenging cleaning projects to industrious contractors. As they’re right across the street from Noah’s, they usually talked about the project over **coffee and bagels** at Noah’s before handing off the item to be cleaned. The contractors would pick up the tab and expense it, along with their cleaning supplies.

“So this rug was apparently one of those special projects. **The claim ticket said ‘2017 JP’. ‘2017’ is the year the item was brought in, and ‘JP’ is the initials of the contractor.**

“But they stopped outsourcing a few years ago, and don’t have contact information for any of these workers anymore.”

Sarah first seemed hopeless, and then glanced at the USB drive you had just put back in her hand. She said, “I know it’s a long shot, but is there any chance you could find their phone number?”

In [30]:
data = get_data(data_path=Path('./../data/5784/'))
df_customers = data["customers"]
df_orders = data["orders"]
df_orders_items = data["orders_items"]
df_products = data["products"]

In [24]:
def get_initials(name: str) -> str:
    return "".join(s[0].upper() for s in name.split(" "))

In [45]:
df_customer_initials = (
    df_customers
    .select(
        pl.col("customerid"),
        pl.col("phone"),
        pl.col("name").map_elements(lambda s: get_initials(name=s)).alias("initials")
    )
    .filter(
        pl.col("initials") == "JP"
    )
)

df_orders_2017 = df_orders.select(pl.col("customerid"), pl.col("ordered").dt.year().alias("year"), pl.col("orderid")).filter(pl.col("year") == 2017)

df_joined = (
    df_customer_initials
    .join(df_orders_2017, on="customerid")
    .join(df_orders_items.select(pl.col("orderid"), pl.col("sku")), on="orderid")
    .join(df_products.select(pl.col("sku"), pl.col("desc")).filter(pl.col("sku").str.contains("DLI")), on="sku")
)

df_joined.filter(pl.col("desc").str.contains("Coffee")).get_column("phone").item()

'332-274-4185'